In [ ]:
import pandas as pd
from geopy.geocoders import Nominatim
from time import sleep

# 初始化 Nominatim 地理编码器
geolocator = Nominatim(user_agent="my_geocoder", timeout=10)

# 读取你的 CSV 文件
data = pd.read_csv("../data/curated/property/property 3.csv")

# 创建一个函数来处理地址地理编码，并处理重试
def geocode_address(address, retries=3):
    for attempt in range(retries):
        try:
            location = geolocator.geocode(address)
            if location:
                return location.latitude, location.longitude
            else:
                return None, None
        except Exception as e:
            print(f"Error geocoding {address}: {e}, retrying ({attempt+1}/{retries})...")
            sleep(2)  # 在每次重试之间等待几秒
    return None, None

# 批次处理函数
def batch_geocode(data, batch_size=100, sleep_time=1):
    latitudes = []
    longitudes = []
    
    # 按照批次处理数据
    for i in range(0, len(data), batch_size):
        batch = data[i:i+batch_size]
        print(f"正在处理第 {i // batch_size + 1} 批...")
        
        for address in batch['address']:
            lat, lon = geocode_address(address)
            latitudes.append(lat)
            longitudes.append(lon)
            sleep(sleep_time)  # 每个请求后暂停，遵守 Nominatim 的速率限制
        
        print(f"第 {i // batch_size + 1} 批处理完成。")
    
    return latitudes, longitudes

# 执行批次处理，假设每批次处理 100 条记录
batch_size = 100  # 每批处理 100 条地址
latitudes, longitudes = batch_geocode(data, batch_size=batch_size, sleep_time=2)

# 将地理编码结果加入到数据集中
data['latitude'] = latitudes
data['longitude'] = longitudes

# 保存所有处理结果到同一个 CSV 文件，无论是否成功
csv_output_path = '../data/curated/property/property_addresses_to_lat_lng.csv'
data.to_csv(csv_output_path, index=False)

print(f"所有地址数据已保存到 {csv_output_path}")


In [ ]:
import pandas as pd
from geopy.geocoders import Nominatim
import re
from time import sleep

# 初始化 Nominatim 地理编码器
geolocator = Nominatim(user_agent="my_geocoder", timeout=10)

# 读取 CSV 文件
data = pd.read_csv('../data/curated/property/property_addresses_to_lat_lng.csv')

# 创建一个函数来处理地址地理编码
def geocode_address(address, retries=3):
    for attempt in range(retries):
        try:
            location = geolocator.geocode(address)
            if location:
                return location.latitude, location.longitude
            else:
                return None, None
        except Exception as e:
            print(f"Error geocoding {address}: {e}, retrying ({attempt+1}/{retries})...")
            sleep(2)  # 每次重试间隔2秒
    return None, None

# 去除地址中的数字和符号
def remove_numbers(address):
    return re.sub(r'[^a-zA-Z\s]', '', address).strip()

# 筛选出没有经纬度的地址
no_lat_lng = data[data['latitude'].isna() | data['longitude'].isna()].copy()

# 去除这些地址中的数字和符号
no_lat_lng['cleaned_address'] = no_lat_lng['address'].apply(remove_numbers)

# 对处理后的地址进行地理编码
no_lat_lng['latitude'], no_lat_lng['longitude'] = zip(*no_lat_lng['cleaned_address'].apply(geocode_address))

# 将更新的结果保存到原始数据中
data.update(no_lat_lng[['latitude', 'longitude']])
# 保存所有结果到 CSV 文件
csv_output_path = '../data/curated/property/property_addresses_to_lat_lng.csv'
data.to_csv(csv_output_path, index=False)

print(f"所有地址数据已保存到 {csv_output_path}")

In [ ]:
import pandas as pd

# 加载你的数据集
file_path = '../data/curated/property/property_addresses_to_lat_lng.csv'  # 替换为你的文件路径
data = pd.read_csv(file_path)

# 删除包含空值的行（即 latitude 或 longitude 为 NaN 的行）
cleaned_data = data.dropna(subset=['latitude', 'longitude'])

# 保存清理后的数据到新的 CSV 文件
output_file_path = '../data/curated/property/property_addresses_to_lat_lng.csv'  # 替换为你想要保存的路径
cleaned_data.to_csv(output_file_path, index=False)

print("已删除经纬度为空的行，清理后的数据已保存。")
